In [1]:
LANG = 'es' # or 'es'
TASK = 'gender' # or 'bot_human'

In [2]:
# Parameters
LANG = "es"
TASK = "gender"


In [3]:
import os
from fastai.text import * 

In [4]:
torch.cuda.device_count()

1

In [5]:
proj_path = Path(os.getcwd()).parents[1]

In [6]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [7]:
df = pd.read_csv(data_path/f'all_{TASK}_data.csv')

In [8]:
df.head()

,gender,text
0,female,RT @ISosprisiones: @PP_Asturias @pablocasado_ ...
1,male,RT @ElNacionalWeb: Nos montan en camiones como...
2,female,RT @HistoriaEnFotos: Video de la semana: A vec...
3,female,RT @WinSportsTV: Colombia se ve favorita para ...
4,female,Os acordáis de cuando vuestra madre os chillab...


In [9]:
data_lm = TextLMDataBunch.from_csv(data_path, f'all_{TASK}_data.csv')

In [10]:
data_clas = TextClasDataBunch.from_csv(data_path, f'all_{TASK}_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [11]:
data_lm.save(f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas.save(f'data_clas_export_{TASK}_{LANG}.pkl')

In [12]:
data_lm = load_data(data_path, fname=f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{TASK}_{LANG}.pkl', bs=16)

In [13]:
if LANG == 'es':
    weights_pretrained = 'model-30k-vocab-noqrnn'
    itos_pretrained = 'itos_pretrained'
    pretained_data = (weights_pretrained, itos_pretrained)
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5, pretrained_fnames=pretained_data)
else:
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,4.553052,4.207654,0.340441,01:41
2,4.219388,3.962740,0.354349,01:42
3,4.098762,3.912495,0.355978,01:43
4,3.973577,3.886660,0.357998,01:42
5,3.864366,3.865171,0.360004,01:41
6,3.744723,3.849120,0.361621,01:41
7,3.638474,3.836219,0.363307,01:42
8,3.535268,3.825939,0.365145,01:42
9,3.454362,3.823777,0.365321,01:42
10,3.409867,3.823786,0.365490,01:41


In [14]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
1,3.310106,3.807929,0.368575,01:53
2,3.273537,3.780135,0.372578,01:53
3,3.216732,3.766174,0.375818,01:53
4,3.128077,3.765678,0.378230,01:53
5,3.043905,3.777556,0.378581,01:53
6,2.945120,3.795199,0.378433,01:53
7,2.850388,3.809687,0.379043,01:53
8,2.775710,3.831020,0.379381,01:53
9,2.727722,3.842573,0.379152,01:54
10,2.701296,3.847342,0.379075,01:54


In [15]:
if LANG == 'es':
    test_string = "Nació en 1910"
else:
    test_string = "He was born in 1910"
learn.predict(test_string, n_words=50)

'Nació en 1910 - Franklin Martínez , Alonso , Morales Solá , Goya del Valle y Kicillof . Los niños menores de la edad de edad . xxbos El bajón a ti no te tiro a la gordura lpm xxbos RT'

In [16]:
learn.save_encoder(f'ft_enc_{TASK}_{LANG}')

In [17]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder(f'ft_enc_{TASK}_{LANG}')

In [18]:
data_clas.show_batch()

text,target
xxbos 🐶 🐺 🐱 🐭 🐹 🐰 🐸 🐯 xxunk 🐻 🐷 🐽 xxunk 🐗 🐵 🐒 🐴 🐑 🐘 🐼 🐧 🐦 🐤 🐥 🐣 🐔 🐍 🐢 🐛 🐝 xxunk 🐞 xxunk 🐙 🐚 🐠 🐟 🐬 🐳 🐋 xxunk 🐏 🐀 xxunk 🐅 🐇 xxunk 🐎 🐐 🐓 🐕 🐪 xxunk 🐂 🐲 🐡 xxunk 🐫 🐪 🐆 🐈 🐩 🐾 💐 🌸 🌷 🍀 🌹 🌻,female
xxbos y xxup eso xxup que xxup no xxup alcanzaron a xxup meterle a xxup la ' xxup jep ' xxup aquel xxup oscuro xxup acuerdo xxup xxunk xxup de 14 xxup magistrados xxup adicionales xxup con xxup xxunk xxup específica a xxup militares xxmaj nueva advertencia de xxup cpi por cambios en casos de militares en la xxup jep https : / / t.co / xxunk vía @eltiempo,male
"xxbos @alfanograce xxup esa xxup cosa xxup de xxup descalificar a xxup los xxup demas , xxup que xxup horror ! xxup una xxup señora xxup vieja xxup como xxup vos . xxup xxunk xxup acaso xxup no xxup te xxup enseño xxup buenos xxup modales xxrep 4 ? xxup jaja",male
xxbos xxup rt @diariodecurwen : xxup roba xxup pero xxup hace xxup xxunk xxup pero xxup hace xxup mal xxup la xxup xxunk xxup pero xxup hace xxup una xxup obra xxup que xxup nadie xxup xxunk xxup pero xxup se xxup demora xxup tres xxup años …,male
xxbos # xxunk xxmaj paola xxmaj barrientos salió del comercial del xxmaj banco xxmaj galicia # xxmaj censurada ? xxmaj todos sabemos la postura política de la actriz y sabemos quienes son los dueños del xxmaj banco ... # xxunk https : / / t.co / xxunk,male


In [19]:
learn_class.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,0.656274,0.625571,0.658045,01:55
2,0.651784,0.658497,0.654645,01:57
3,0.663898,0.662047,0.644812,01:56
4,0.660711,0.676940,0.607246,01:54
5,0.654997,0.738465,0.651112,02:00
6,0.656423,0.709219,0.656678,01:56
7,0.661677,0.632961,0.651278,01:59
8,0.641631,0.618923,0.665744,01:56
9,0.650935,0.621882,0.663611,01:59
10,0.647808,0.615506,0.670578,01:54


In [20]:
learn.save(f'model_stage_1_{TASK}_{LANG}')